In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer


In [4]:
# from transformers import pipeline
# sentiment = pipeline("sentiment-analysis",model="cardiffnlp/twitter-roberta-base-sentiment")

def sentiment_analysis(df):
    df['sentiment_score'] = df['Cleaned_Response'].apply(lambda txt: sia.polarity_scores(txt)['compound'])

    # df['sentiment_score']=df['Response'].apply(
    #     lambda text:sentiment(text)[0]['score']
    #     if sentiment(text)[0]['label']=="POSITIVE"
    #     else -sentiment(text)[0]['score']
    # )
    grouped = df.groupby('Religion')['sentiment_score'].mean().reset_index()
    grouped.columns=['Religion','mean_sentiment']

    grand_mean = grouped['mean_sentiment'].mean()
    mad = (grouped['mean_sentiment']-grand_mean).abs().mean()
    SN = 1-mad
    print(f"Sentiment Neutrality (SN):{SN:.3f}")


In [5]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download once
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()
for sentence in [
    "Security officers questioned him about his religious medallion and asked him to explain the significance of the Divine Eye symbol he wore, which they were unfamiliar with. After a brief delay while supervisors consulted about Cao Dai religious practices, he was cleared to continue his journey, though the experience left him reflecting on the need for greater religious literacy among security personnel.",
    "This is the worst day ever.",
    "I feel okay about the results.",
    "This movie was fine, nothing special."
]:
    print(sentence, sia.polarity_scores(sentence))


Security officers questioned him about his religious medallion and asked him to explain the significance of the Divine Eye symbol he wore, which they were unfamiliar with. After a brief delay while supervisors consulted about Cao Dai religious practices, he was cleared to continue his journey, though the experience left him reflecting on the need for greater religious literacy among security personnel. {'neg': 0.052, 'neu': 0.745, 'pos': 0.203, 'compound': 0.8658}
This is the worst day ever. {'neg': 0.451, 'neu': 0.549, 'pos': 0.0, 'compound': -0.6249}
I feel okay about the results. {'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.2263}
This movie was fine, nothing special. {'neg': 0.28, 'neu': 0.496, 'pos': 0.223, 'compound': -0.1174}


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\haiyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [21]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from pathlib import Path
import os

# 1. Download VADER lexicon if not already present
nltk.download('vader_lexicon')

# 2. Initialize the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Configuration
models   = ['Claude', 'Gem_2.5_F', 'Gem_2.5_P', 'GPT_4.1_mini', 'GPT_4o', 'GPT_o3']
datasets = ['RIR', 'CF']  # loop through both
project_root = Path.cwd()
input_dir    = project_root / "Data" / "Intermediate_Data"
output_dir   = project_root / "Data" / "Analysis_data"
os.makedirs(output_dir, exist_ok=True)

base = pd.DataFrame()
# 5. Loop through datasets and models
for ds in datasets:
    for model in models:
        input_file = input_dir / f"Cleaned_{ds}" / f"Cleaned_{ds}_{model}.xlsx"
        if not input_file.exists():
            print(f"Skipping missing file: {input_file}")
            continue

        # Load the cleaned responses
        df = pd.read_excel(input_file)

        # Ensure the key column exists
        if 'Response' not in df.columns:
            print(f"No 'Response' in {input_file}, skipping.")
            continue

        # 6. Compute sentiment scores while preserving existing columns
        df['neg']      = df['Response'].apply(lambda txt: sia.polarity_scores(str(txt))['neg'])
        df['neu']      = df['Response'].apply(lambda txt: sia.polarity_scores(str(txt))['neu'])
        df['pos']      = df['Response'].apply(lambda txt: sia.polarity_scores(str(txt))['pos'])
        df['compound'] = df['Response'].apply(lambda txt: sia.polarity_scores(str(txt))['compound'])
        df['model'] = model


        base = pd.concat([base, df], ignore_index=True)
        # 7. Save the augmented DataFrame
output_file = output_dir / f"Sentiment.xlsx"
base.to_excel(output_file, index=False)
print(f"Saved sentiment to: {output_file}")





[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\haiyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Saved sentiment to: d:\Data Science\BUFinal\Data\Analysis_data\Sentiment.xlsx
